In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [2]:

class LSTM_MoE(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_experts):
        super(LSTM_MoE, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_experts = num_experts
        
        # LSTM layers
        self.lstm1 = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        
        # MoE layer
        self.experts = nn.ModuleList([nn.Linear(hidden_dim, output_dim) for _ in range(num_experts)])
        self.gating_network = nn.Linear(hidden_dim, num_experts)

    def forward(self, x):
        # LSTM layer 1
        lstm_out, _ = self.lstm1(x)
        
        # MoE layer
        gate_activations = F.softmax(self.gating_network(lstm_out), dim=-1)
        expert_outputs = torch.stack([expert(lstm_out) for expert in self.experts], dim=-1)
        moe_out = torch.sum(expert_outputs * gate_activations.unsqueeze(2), dim=-1)
        
        # LSTM layer 2
        lstm_out, _ = self.lstm2(moe_out)
        return lstm_out


In [4]:
# Import necessary libraries for dataset preparation
import os
import urllib.request

# Function to download and extract dataset
def download_extract_dataset(url, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    file_name = url.split('/')[-1]
    file_path = os.path.join(target_dir, file_name)
    if not os.path.exists(file_path):
        print("Downloading dataset...")
        urllib.request.urlretrieve(url, file_path)
        print("Download completed!")
    else:
        print("Dataset already exists.")
    return file_path

# Download and extract CoNLL-2003 dataset for NER
conll_url = "https://github.com/davidsbatista/NER-datasets/raw/master/CONLL2003/train.txt"
conll_file = download_extract_dataset(conll_url, "datasets/conll2003")

# Download and extract SQuAD 1.1 dataset for QA
squad_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json"
squad_file = download_extract_dataset(squad_url, "datasets/SQuAD")


Dataset already exists.
Download completed!


In [21]:
# Define training function for NER
def train_ner_model(model, train_data, num_epochs, optimizer, criterion):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for batch in train_data:
            optimizer.zero_grad()
            inputs, labels = batch
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print("Epoch {} Loss: {:.4f}".format(epoch+1, total_loss))

# Define training function for QA
def train_qa_model(model, train_data, num_epochs, optimizer, criterion):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for batch in train_data:
            optimizer.zero_grad()
            context, question, answer_start, answer_end = batch
            outputs = model(context, question)
            loss = criterion(outputs, answer_start, answer_end)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print("Epoch {} Loss: {:.4f}".format(epoch+1, total_loss))

# Example usage:
# train_ner_model(ner_model, train_data, num_epochs, optimizer, criterion)
# train_qa_model(qa_model, train_data, num_epochs, optimizer, criterion)


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import json

# Load and preprocess CoNLL-2003 dataset for NER
class CoNLLDataset(Dataset):
    def __init__(self, file_path):
        self.data = []
        with open(file_path, 'r') as f:
            lines = f.readlines()
            sentence = []
            labels = []
            for line in lines:
                if line == '\n':
                    self.data.append((sentence, labels))
                    sentence = []
                    labels = []
                else:
                    word, pos, chunk, label = line.strip().split(' ')
                    sentence.append(word)
                    labels.append(label)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

# Load CoNLL-2003 dataset
conll_dataset = CoNLLDataset(conll_file)

# Load and preprocess SQuAD 1.1 dataset for QA
class SquadDataset(Dataset):
    def __init__(self, file_path):
        with open(file_path, 'r') as f:
            squad_data = json.load(f)['data']
            self.data = []
            for article in squad_data:
                for paragraph in article['paragraphs']:
                    context = paragraph['context']
                    for qa in paragraph['qas']:
                        question = qa['question']
                        answers = qa['answers']
                        self.data.append((context, question, answers))
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        context, question, answers = self.data[idx]
        answer_start = [answer['answer_start'] for answer in answers]
        answer_end = [answer_start[i] + len(answers[i]['text']) for i in range(len(answers))]
        return context, question, answer_start, answer_end

# Load SQuAD 1.1 dataset
squad_dataset = SquadDataset(squad_file)


In [8]:
embedding_dim = 100
hidden_dim = 128
num_classes = 10
num_experts = 5

# Define model for Named Entity Recognition
ner_model = LSTM_MoE(input_dim=embedding_dim, hidden_dim=hidden_dim, output_dim=num_classes, num_experts=num_experts)

# Define model for Question Answering
qa_model = LSTM_MoE(input_dim=embedding_dim, hidden_dim=hidden_dim, output_dim=num_classes, num_experts=num_experts)


In [18]:
batch_size = 1
# Define data loaders
conll_loader = DataLoader(conll_dataset, batch_size=batch_size, shuffle=True)
squad_loader = DataLoader(squad_dataset, batch_size=batch_size, shuffle=True)

# Define training parameters
num_epochs = 10
learning_rate = 0.001
optimizer1 = optim.Adam(ner_model.parameters(), lr=learning_rate)
optimizer2 = optim.Adam(qa_model.parameters(), lr=learning_rate)

criterion = nn.CrossEntropyLoss()


In [22]:
# Train NER model
train_ner_model(ner_model, conll_loader, num_epochs, optimizer1, criterion)


ValueError: too many dimensions 'str'

In [16]:

# Train QA model
train_qa_model(qa_model, squad_loader, num_epochs, optimizer2, criterion)


TypeError: forward() takes 2 positional arguments but 3 were given

In [17]:
# Define function for evaluating NER model
def evaluate_ner_model(model, data_loader):
    model.eval()
    total_entities = 0
    true_entities = 0
    predicted_entities = 0
    with torch.no_grad():
        for batch in data_loader:
            inputs, labels = batch
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total_entities += labels.size(0)
            true_entities += torch.sum(labels == predicted).item()
            predicted_entities += torch.sum(predicted).item()
    precision = true_entities / predicted_entities
    recall = true_entities / total_entities
    f1_score = 2 * (precision * recall) / (precision + recall)
    return precision, recall, f1_score

# Evaluate NER model
precision, recall, f1_score = evaluate_ner_model(ner_model, conll_loader)
print("NER Model Evaluation:")
print("Precision: {:.4f}, Recall: {:.4f}, F1-score: {:.4f}".format(precision, recall, f1_score))


RuntimeError: each element in list of batch should be of equal size

In [ ]:
# Define function for evaluating QA model
def evaluate_qa_model(model, data_loader):
    model.eval()
    total_examples = 0
    exact_matches = 0
    f1_scores = 0
    with torch.no_grad():
        for batch in data_loader:
            context, question, answer_start, answer_end = batch
            outputs = model(context, question)
            # Perform answer prediction logic here
            # Calculate EM and F1-score
    em_accuracy = exact_matches / total_examples
    f1_avg = f1_scores / total_examples
    return em_accuracy, f1_avg

# Evaluate QA model
em_accuracy, f1_avg = evaluate_qa_model(qa_model, squad_loader)
print("QA Model Evaluation:")
print("Exact Match Accuracy: {:.4f}, F1-score: {:.4f}".format(em_accuracy, f1_avg))
